<a href="https://colab.research.google.com/github/Prashu7487/RUL-Prediction/blob/main/Reading_MATLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ashiquem/vault/blob/master/python/Matlab%20Data%20Demo.ipynb)



### Loading and inspecting the file

In [ ]:
from scipy.io import loadmat
import pandas as pd

In [ ]:
annots = loadmat('B0005.mat')
type(annots)

dict

In [ ]:
print(annots.keys())
annots['__header__']

dict_keys(['__header__', '__version__', '__globals__', 'B0005'])


b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Wed Oct 01 15:50:14 2008'

In [ ]:
print(annots['__globals__'])
annots['__version__']

[]


'1.0'

In [ ]:
print(type(annots['B0005']))
print(annots['B0005'].ndim)     #for dimension of annots array
# B0005= annots['B0005'].items()
# B0005.keys()
# Since this array is 2D, so print starting 10 values of each 1D entry of this array
# annots['B0005'][0,0:10]
print(annots['B0005'].dtype)
print(annots['B0005'].shape)

<class 'numpy.ndarray'>
2
[('cycle', 'O')]
(1, 1)


### Flattening and loading into lists

In [ ]:
# convert str to datatime 
from datetime import datetime
def convert_to_time(hmm):
    year, month, day, hour, minute, second = int(hmm[0]), int(hmm[1]), int(hmm[2]), int(hmm[3]), int(hmm[4]), int(hmm[5])
    return datetime(year=year, month=month, day=day, hour=hour, minute=minute, second=second)


# load .mat data
def loadMat(matfile):
    data = scipy.io.loadmat(matfile)
    filename = matfile.split("/")[-1].split(".")[0]
    col = data[filename]
    col = col[0][0][0][0]
    size = col.shape[0]

    data = []
    for i in range(size):
        k = list(col[i][3][0].dtype.fields.keys())
        d1, d2 = {}, {}
        if str(col[i][0][0]) != 'impedance':
            for j in range(len(k)):
                t = col[i][3][0][0][j][0];
                l = [t[m] for m in range(len(t))]
                d2[k[j]] = l
        d1['type'], d1['temp'], d1['time'], d1['data'] = str(col[i][0][0]), int(col[i][1][0]), str(convert_to_time(col[i][2][0])), d2
        data.append(d1)

    return data
  
# get capacity data
def getBatteryCapacity(Battery):
    cycle, capacity = [], []
    i = 1
    for Bat in Battery:
        if Bat['type'] == 'discharge':
            capacity.append(Bat['data']['Capacity'][0])
            cycle.append(i)
            i += 1
    return [cycle, capacity]


# get the charge data of a battery
def getBatteryValues(Battery, Type='charge'):
    data=[]
    for Bat in Battery:
        if Bat['type'] == Type:
            data.append(Bat['data'])
    return data

In [ ]:
Battery_list = ['B0005', 'B0006', 'B0007', 'B0018']
# dir_path = 'datasets/NASA/'
dir_path = '/content/'

Battery = {}
for name in Battery_list:
    print('Load Dataset ' + name + '.mat ...')
    path = dir_path + name + '.mat'
    data = loadMat(path)
    Battery[name] = getBatteryCapacity(data)
